# Get timeseries of F0s
- Measures every 10 ms
- Needs to be extended to multiple wav files
- Need to add filename somehow as a feature, to keep track

In [1]:
import os
import pandas as pd
import numpy as np
from Tkinter import *
import tkSnack
root = Tk()
tkSnack.initializeSnack(root)

import matplotlib.pyplot as plt
%matplotlib notebook

from sklearn.externals.joblib import Memory
memory = Memory(cachedir="cache")

C:\Python27\lib\site-packages\IPython\kernel\__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [3]:
##from pylab import *
@memory.cache
def getSound(fname_list):
    pitch_list = []
    for item in fname_list:
        utterance = tkSnack.Sound()
        utterance.read(item)
        utterance.convert(channels=1)
        pitch_tuple = utterance.pitch(method="ESPS", windowlength = 0.010)
        pitch_list.append(pitch_tuple)
        utterance.destroy()
    return pitch_list



path = "c:/Python27/qp2_py/dev50wav/"
#path = "c:/Python27/qp2_py/data/"
allWavs = os.listdir(path)
#allTxtgridsCOMB = [item for item in allFiles if item[0]== "C"]
#allWavs = [item for item in allFiles if item[0] !="C"]

allPaths = []
for item in allWavs:
    total_path = path+item
    allPaths.append(total_path)


    
#fname = [non_fname, native_fname]
pitch_list = getSound(allPaths)

In [4]:
print len(pitch_list)

51


# To Do:
- In this cell, get all of the name of files for baseline / test v train split

In [5]:
### Make a list of lists of F0 points

all_f0s_list = []
for item in pitch_list:
    just_f0_list = [j[0] for j in item]
    all_f0s_list.append(just_f0_list)


    
print len(all_f0s_list)


## output is [[series of f0s for one file][series of f0s for next file]] etc 




51


## Make TextGrid Objects
- Also needs to be extended to multiple files
- Edit this if you want to look at tiers other than IPU tiers (like Question Tiers)

In [6]:

import tgt
#non_txtgrd = "c:/Python27/qp2_py/txtgrids/COMBINE_p452p453-part2_ch1.textgrid"
#native_txtgrd = "c:/Python27/qp2_py/txtgrids/p226p227-part2_ch2.textgrid"
#all_txtgrids = [non_txtgrd, native_txtgrd]

allTxtPaths = []
path = "c:/Python27/qp2_py/dev50txtgrid/"
allTxtGrids = os.listdir(path)
for item in allTxtGrids:
    allTxtPaths.append(path+item)


checkGrid = allTxtPaths[0]
print len(allTxtPaths)

51


In [7]:
grid_obj= tgt.read_textgrid(checkGrid)
grid_obj.get_tier_names()


[u'BF',
 u'WORD',
 u'LONG_WORD',
 u'LONG_IPUs',
 u'IPU_WORDs',
 u'LAUGHTER',
 u'QUESTIONS']

# Combine TextGrids with F0s
- Has been extended?

In [8]:
#annotations are a list of interval objects
inter = []
segmented_f0s = []

for i in range(len(allTxtPaths)):
    grid_obj = tgt.read_textgrid(allTxtPaths[i])
    word = grid_obj.get_tier_by_name('IPU_WORDs')
    ipus = word.annotations
    length = len(ipus)
    for j in range(length):
        start = ipus[j].start_time
        end = ipus[j].end_time
        word = ipus[j].text
        #print start, end, word
        #multiply to get the correct index
        idx_start = int(start*100)
        idx_end = int(end*100)
        #print idx_start, idx_end
        f0s = all_f0s_list[i][idx_start:idx_end]
        entry = (start, end, f0s, word)
        inter.append(entry)
    segmented_f0s.append(inter)
    inter = []

    
    
print len(segmented_f0s)





## make this a loop
#first_word = word.annotations[0]
#print first_word.start_time
#print first_word.end_time
#print first_word.text
#returns a list of lists --> inner list is a tuple of (starttime, endtime, [listofF0series],"textstring")

51


In [9]:
###NOTES ABOUT THIS FUNCTION: 
## Extended for large data lists (2/16/2016)
## log normalizes each F0 point in the IPU, then calculates min, max, and std of the normalized F0s
## also calculates mean and the z-normalized mean of the logF0 points per IPU
## currently (2/15/2016) returns a list of (ipuString, [seconds, minF0, maxF0, std, meanF0, znorm_meanF0, whisper])
## Goal is to output format that's good for sklearn
## NOTE:  For whispered ipus, all values are "NaN"
## NOTE: For one F0 point ipus, min, max, and mean F0 features are log of the F0 point, std&znorm are NaN, whisper = 0


## QUESTIONS FOR ANDREW (2/3/2016):
### z-normalized is returning between 1 and -1, is that okay?
## Is the normalization I'm doing okay
## can I z-score normalize already log normalized f0 means?
## QUESTIONS 2/15/2016
## is my zscore normalization correct?
## handling whisperes and f0 points okay?



#### segmented_f0s

#segmented_f0s is a list [start_time, end_time, [series of F0 measures every 10ms], "ipu_string"]
import scipy
import math
from scipy import stats

feature_set = []
intermediate = []

for i in range(len(segmented_f0s)):
    current = segmented_f0s[i]
    for item in current:
    ## assign text, calculate length    
        txt = str(item[3])
        start_time = item[0]
        end_time = item[1]
        time = end_time-start_time

        f0s = item[2]
        ## log of all valid f0 points
        logF0s = [math.log(f0) for f0 in f0s if f0 !=0]
        ##### if no F0 points
        if bool(logF0s) == False:
            minF0 = "NaN"
            maxF0 = "NaN"
            std = 0
            meanF0 = 0
            znorm_mean = 0
            whisper = 1
        ## if only 1 F0 value    
        elif len(logF0s) == 1:
            oneVal = math.log(logF0s[0])
            minF0 = oneVal
            maxF0 = oneVal
            std = 0
            meanF0 = oneVal
            znorm_mean = 0
            whipser = 0
        
        else:
            minF0 = min(logF0s)
            maxF0 = max(logF0s)
            meanF0 = sum(logF0s)/len(logF0s)
            std = scipy.stats.tstd(logF0s)
            for point in logF0s:
                znorm_mean = (point-meanF0)/std
            whisper = 0
        #print "min:%f, max:%f, std:%f, mean:%f, znormMean:%f" %(minF0, maxF0, std, meanF0, znorm_mean)

        features = [time, minF0, maxF0, std, meanF0, znorm_mean, whisper]
        set_tup = (txt, features)
        intermediate.append(set_tup)
    feature_set.append(intermediate)
    intermediate = []

print len(feature_set)
#for i in range(len(feature_set)):
#    print len(feature_set[i])

51


# Get rid of silence & unicode

In [10]:
done_features = []
inter = []
for ipu in feature_set:
    for features in ipu:
        if features[0] == "silent":
            pass
        else:
            inter.append(features)
    done_features.append(inter)
    inter = []

print len(done_features)
print len(done_features[0])

51
522


# Add Pronouncing Features

In [11]:
####Calculates syllables for each IPU
## NOTE: Assumes words not in the dictionary are all monosyllabic (assumption based on um, mm, and false starts)
### But this is definitely not accurate, there will be names that are wrong too.
## Extended for large data (2/16/2016)

import pronouncing


def getWeirdSyllables(text):
    """Gets syllables count for IPUs that contain OOVs.  NOTE:  OOVs always return monosyllabic"""
    syl_sum = 0
    checkit = text.split()
    for word in checkit:
        valid = pronouncing.phones_for_word(word)
        #print valid
        if valid:
            syl = sum([pronouncing.syllable_count(p)for p in valid[0]])
        else:
            syl = 1
        syl_sum = syl_sum+syl
    return syl_sum


######### 5/19 rewrote this but did not check ##########
@memory.cache    
def getPhones(done_features):
    syl_list = []
    intermediate = []
    for features in done_features:    
        for item in features:
            text = item[0]
            text = text.lower()
            try:
                phones = [pronouncing.phones_for_word(p)[0] for p in text.split()]
                syl = sum([pronouncing.syllable_count(p) for p in phones])

            except IndexError:
                syl = getWeirdSyllables(text)
            intermediate.append(syl)
        syl_list.append(intermediate)
        intermediate = []
    return syl_list


syl_list = getPhones(done_features)

print len(syl_list)
#for i in range(len(syl_list)):
#    print len(syl_list[i])


________________________________________________________________________________
[Memory] Calling __main__-C%3A-Users-Rachel-Documents-Grad_Stuff-QP2-code-__ipython-input__.getPhones-alias...
getPhones-alias([ [ ('OKAY', [...]),
    ('UM', [...]),
    ('SHOULD I START AGAIN', [...]),
    ('OKAY', [...]),
    ('UH', [...]),
    ('UH', [...]),
    ('BACK TO THIS TENNIS RACQUET OKAY', [...]),
    ('UM', [...]),
    ("UH SO YOU'VE YOU'VE", [...]),
    ('HAVE YOU EVER PLAYED TENNIS', [...]),
    ('UH HOW OLD WERE YOU', [...]),
    ('HIGH SCHOOL', [...]),
    ('UH WAS THIS LIKE A ONE TIME DEAL', [...]),
    ('OK DO DO UH DOES ANYONE IN YOUR FAMILY PLAY TENNIS', [...]),
    ('OKAY', [...]),
    ('UM', [...]),
    ('ALRIGHT', [...]),
    ('UH HOW MANY ROOMMATES DO YOU HAVE', [...]),
    ('TWO', [...]),
    ('WHAT ARE THEIR', [...]),
    ('NAMES', [...]),
    ('NIGERIA AND ZACH', [...]),
    ('UM', [...]...)
_______________________________________________getPhones-alias - 989.3s, 16.5min
51


In [17]:
## replaces time with seconds per syllable
## feature_set is now done for a baseline, for many files (3/25/2016)

num_datapoint = len(done_features)

for num in range(num_datapoint):
    for i in range(len(done_features[num])):
        #print done_features[num][i][1]
        length = float(done_features[num][i][1][0])
        ##feature set [each featurelist][each ipu][list of features][seconds]
        syl_num = syl_list[num][i] #get the # of syllables
        #print length
        #print syl_num
        syl_sec =  syl_num/length
        #print "SPACE"
        done_features[num][i][1][0]= syl_sec #replace with syllables per second
        #print done_features[num][i][1][0]




## Build in quick check
for num in range(len(done_features)):
   for i in range(10):
        print done_features[num][i]

('OKAY', [4.545454545454541, 3.9805274458845017, 4.3385405817750415, 0.13031909775532191, 4.189567739348641, -1.6040649226763302, 0])
('UM', [1.9230769230769247, 4.036843327856896, 4.884392541609571, 0.2468998146177952, 4.358264899241548, 1.6620307033086545, 0])
('SHOULD I START AGAIN', [6.944444444444446, 4.482975051332292, 4.905337054964194, 0.09902424823110248, 4.675694031740486, 2.3190584864402948, 0])
('OKAY', [6.896551724137931, 4.685640869787632, 5.295125556335249, 0.20534871732916818, 4.876461268225521, -0.60075698379978382, 0])
('UH', [2.1276595744680864, 4.149504558741698, 5.095586869536976, 0.16878255030227396, 4.688794537849597, 2.4101563281207139, 0])
('UH', [0.8620689655172413, 3.9990355268263005, 5.205517757901706, 0.32075851807972666, 4.299041798300083, 1.5654377138374416, 0])
('BACK TO THIS TENNIS RACQUET OKAY', [7.317073170731705, 4.160501707715904, 6.1583703170809105, 0.45961502550831806, 4.959343588225798, -0.51133888250392445, 0])
('UM', [1.4705882352941182, 4.3355

## Make IPU based features speaker based features
time, minF0, maxF0, std, meanF0, znorm_mean, whisper

- number ipus per speaker
- avg syls per ipu per speaker
- min f0 per speaker
- max f0 per speaker
- mean f0 per speaker
- mean std per speaker
- avg znorm mean per speaker
- number of whispered ipus
- anything else to add?


In [18]:
syl_per_speaker = []
minf0_per_speaker = []
maxF0_per_speaker = []
meanf0_per_speaker = []
meanStd_per_speaker = []
meanZnorm_per_speaker = []
whispers_per_speaker = []




syl_sum =0
meanf0_sum = 0
meanstd_sum = 0
meanZnorm_sum = 0
whisper_tot = 0

mins = []
maxs = []

[7.692307692307751, 4.778437317477203, 5.349490509827351, 0.17253538730659007, 4.908782448256691, -0.087913055167218049, 0]


for speaker_ipus in done_features:
    num_ipus = len(speaker_ipus)
    for item in speaker_ipus:
        syls = item[1][0]
        #print syls
        min_check =item[1][1]
        if min_check !="NaN":
            mins.append(min_check)
        max_check = item[1][2]
        if max_check != 'NaN':
            maxs.append(max_check)
        meanf0 = item[1][3]
        #print meanf0
        std = item[1][4]
        #print std
        znorm = item[1][5]
        #print znorm
        whisper = item[1][6]
        #print whisper
        ### calculate ######
        syl_sum = syl_sum+syls
        meanf0_sum = meanf0+meanf0_sum
        meanstd_sum = std+meanstd_sum
        meanZnorm_sum = znorm+meanZnorm_sum
        whisper_tot = whisper + whisper_tot
    
    #print maxs
    #calculate & reset
    syl_speaker = syl_sum/num_ipus
    minF0 = min(mins)
    maxF0 = max(maxs)
    meanF0_speaker = meanf0_sum / num_ipus
    meanStd_speaker = meanstd_sum/ num_ipus
    meanZnorm_speaker = meanZnorm_sum/num_ipus
    
    print syl_speaker, minF0, maxF0, meanF0_speaker, meanStd_speaker, meanZnorm_speaker, whisper_tot
    #### put them in out feature vectors
    syl_per_speaker.append(syl_speaker)
    minf0_per_speaker.append(minF0)
    maxF0_per_speaker.append(maxF0)
    meanf0_per_speaker.append(meanF0_speaker)
    meanStd_per_speaker.append(meanStd_speaker)
    meanZnorm_per_speaker.append(meanZnorm_speaker)
    whispers_per_speaker.append(whisper_tot)
    
    #### reset for next run###
    syl_sum =0
    meanf0_sum = 0
    meanstd_sum = 0
    meanZnorm_sum = 0
    whisper_tot = 0
    mins = []
    maxs = []
    

        

3.72622058684 3.97943227708 6.29383921633 0.24772739308 4.80471845269 -0.150966191916 8
3.36641283408 3.97932600384 6.28894537081 0.247534077193 5.01700446871 -0.289292092882 0
2.98266186394 3.98300652934 6.2814539075 0.241352540257 5.1197769973 -0.406997456401 0
3.80374611596 3.97805891338 5.73546626061 0.140937044119 4.73891400954 -0.137249170827 2
4.25264175644 3.9810371983 6.28760406336 0.140054159334 4.75466246913 -0.226808001957 6
3.34536073271 3.97580533462 6.27625287985 0.282529060254 5.07904932492 0.0343588813632 3
3.80302899171 3.98017786923 6.18808400929 0.144883593135 4.7622780289 0.189346890203 5
4.50620292669 3.978749792 6.24570392329 0.100663801898 4.51316860043 0.153678304765 1
3.15604438048 3.98617772288 6.28805106331 0.166906852373 5.13686532372 -0.302337373922 0
2.94851581462 3.9799344615 6.24438423331 0.213999347533 5.09800298321 -0.229734857701 3
3.37318861587 3.97857121024 6.28143575317 0.190770999495 5.26510670377 0.370598573771 0
5.16735210846 3.99000819202 5.78

In [19]:
### This is where you will make your dataframes 
import pandas as pd
#print len(syl_per_speaker)
#print minf0_per_speaker
#print maxF0_per_speaker
#print meanf0_per_speaker
#print meanStd_per_speaker
#print meanZnorm_per_speaker
#print len(whispers_per_speaker)

In [20]:
import pandas as pd
import re

features = pd.DataFrame(syl_per_speaker, columns=["Syl_Per_Speaker"])
features["minF0"] = minf0_per_speaker
features["maxF0"]=maxF0_per_speaker
features["meanF0"] = meanf0_per_speaker
features["meanStd"] = meanStd_per_speaker
features["meanZnorm"] = meanZnorm_per_speaker
features["Total_Whispered"] = whispers_per_speaker

print features


    Syl_Per_Speaker     minF0     maxF0    meanF0   meanStd  meanZnorm  \
0          3.726221  3.979432  6.293839  0.247727  4.804718  -0.150966   
1          3.366413  3.979326  6.288945  0.247534  5.017004  -0.289292   
2          2.982662  3.983007  6.281454  0.241353  5.119777  -0.406997   
3          3.803746  3.978059  5.735466  0.140937  4.738914  -0.137249   
4          4.252642  3.981037  6.287604  0.140054  4.754662  -0.226808   
5          3.345361  3.975805  6.276253  0.282529  5.079049   0.034359   
6          3.803029  3.980178  6.188084  0.144884  4.762278   0.189347   
7          4.506203  3.978750  6.245704  0.100664  4.513169   0.153678   
8          3.156044  3.986178  6.288051  0.166907  5.136865  -0.302337   
9          2.948516  3.979934  6.244384  0.213999  5.098003  -0.229735   
10         3.373189  3.978571  6.281436  0.190771  5.265107   0.370599   
11         5.167352  3.990008  5.785231  0.063137  1.669338   0.158235   
12         4.406870  3.977100  5.94059

In [22]:
##### 4/20 this works now #########

label_csv = pd.read_csv("c:/Python27/qp2_py/labels/devlabels50.csv")



features["labels"] = label_csv["Language"]
print features



    Syl_Per_Speaker     minF0     maxF0    meanF0   meanStd  meanZnorm  \
0          3.726221  3.979432  6.293839  0.247727  4.804718  -0.150966   
1          3.366413  3.979326  6.288945  0.247534  5.017004  -0.289292   
2          2.982662  3.983007  6.281454  0.241353  5.119777  -0.406997   
3          3.803746  3.978059  5.735466  0.140937  4.738914  -0.137249   
4          4.252642  3.981037  6.287604  0.140054  4.754662  -0.226808   
5          3.345361  3.975805  6.276253  0.282529  5.079049   0.034359   
6          3.803029  3.980178  6.188084  0.144884  4.762278   0.189347   
7          4.506203  3.978750  6.245704  0.100664  4.513169   0.153678   
8          3.156044  3.986178  6.288051  0.166907  5.136865  -0.302337   
9          2.948516  3.979934  6.244384  0.213999  5.098003  -0.229735   
10         3.373189  3.978571  6.281436  0.190771  5.265107   0.370599   
11         5.167352  3.990008  5.785231  0.063137  1.669338   0.158235   
12         4.406870  3.977100  5.94059

In [ ]:
#### 4/20/16 this is no longer necessary

#### Creates CSV dataframe, gets all text grid and wav files names
label_csv = pd.read_csv("c:/Python27/qp2_py/labels/Participants_info_200-563.csv")
path = "c:/Python27/qp2_py/wav_baseline/"
allFiles = os.listdir(path)
allTxtgridsCOMB = [item for item in allFiles if item[0]== "C"]
allWavs = [item for item in allFiles if item[0] !="C"]

allTxtgrids = []
for item in allTxtgridsCOMB:
    outlist = item.split('COMBINE_')
    name = outlist[1]
    allTxtgrids.append(name)

    
allTxtgrids.sort()
allWavs.sort()


wavBaseline = []
for item in allWavs:
    if re.search('.*-baseline_.*', item):
        wavBaseline.append(item)

### Makes a list of tuples (IDnum, channel number they have)
IDs = []
for item in wavBaseline:
    searched = re.search('p(\d.*)-', item)
    outnum = searched.group(1)
    ch_searched = re.search('.*_(ch\d).*', item)
    channel = ch_searched.group(1)
    entry = (outnum, channel)
    IDs.append(entry)


## Puts tuples (ID, channel) into the label dataframe
id_channels = []
for i in range(len(label_csv['PID'])):
    ID = label_csv['PID'][i]
    for item in IDs:
        if int(item[0]) == ID:
            Found = item
            break
        else:
            Found = False
    if Found:
        id_channels.append(Found)
    else:
        id_channels.append("NA")
        
label_csv["ID_Channels"] =  id_channels 
print label_csv

In [ ]:
data_path = "c:/Python27/qp2_py/small_for_test/"
allFiles = os.listdir(data_path)
allCurrentWavs = [item for item in allFiles if item[0] !="C"]

#print allCurrentWavs

small_test =  label_csv[4:10]


add1 = label_csv.loc[65]
add2 = label_csv.loc[186]
add3 = label_csv.loc[217]
add4 = label_csv.loc[216]


small_test = small_test.append(add1)
small_test = small_test.append(add2)
small_test = small_test.append(add3)
small_test = small_test.append(add4)
#print small_test

label_list = []
x = small_test["Language"]
for i in x:
    label_list.append(i)

print label_list
features["labels"] = label_list


In [ ]:
print features

In [ ]:
data_path = "c:/Python27/qp2_py/data/"
allFiles = os.listdir(data_path)
allCurrentWavs = [item for item in allFiles if item[0] !="C"]

### not sure what to do about this code
for wav in allCurrentWavs:
    searched = re.search('.*_(ch\d).*', wav)
    channel = searched.group(1)
    if re.search("p(\d+)p(\d+)-(.*)_(ch\d)", wav):
        nonBases = re.search("p(\d+)p(\d+)-(.*)_(ch\d)", wav)
        id1, id2 = nonBases.group(1), nonBases.group(2)
        #print id1, id2, channel
    for label in label_csv["ID_Channels"]:
        if type(label)== tuple:
            y = label[0]
            if id1 == y:
                take = label
                break
    #print wav, take




small_test =  label_csv[4:10]
test = (label_csv.loc[1])




print small_test
small_labels = small_test["Language"]


#features["labels"] = small_labels



In [24]:


labels = (features[features.columns[7]])
x_train =(features[features.columns[:7]])
print labels

X_train= x_train[:37]
X_test = x_train[37:51]

print X_test


y_train = labels[:37]
y_test = labels[37:51]

#print X_train, y_train, X_test, y_test




0      English
1      English
2      English
3     Mandarin
4     Mandarin
5     Mandarin
6     Mandarin
7      English
8      English
9     Mandarin
10     English
11     English
12     English
13     English
14     English
15    Mandarin
16     English
17     English
18     English
19     English
20     English
21     English
22     English
23    Mandarin
24     English
25     English
26     English
27     English
28     English
29    Mandarin
30     English
31    Mandarin
32    Mandarin
33    Mandarin
34    Mandarin
35    Mandarin
36    Mandarin
37    Mandarin
38    Mandarin
39    Mandarin
40    Mandarin
41    Mandarin
42    Mandarin
43    Mandarin
44    Mandarin
45    Mandarin
46    Mandarin
47    Mandarin
48    Mandarin
49     English
50     English
Name: labels, Length: 51, dtype: object
    Syl_Per_Speaker     minF0     maxF0    meanF0   meanStd  meanZnorm  \
37         3.112750  3.984530  6.299079  0.252490  5.300734  -0.055155   
38         3.909264  3.994113  6.292385  0.1947

In [26]:
from sklearn import svm
from sklearn import linear_model
#classifier = linear_model.LogisticRegression()
classifier = svm.LinearSVC()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
predicts = classifier.predict(X_test)

print score, predicts


0.0714285714286 ['English' 'English' 'English' 'English' 'English' 'Mandarin' 'English'
 'English' 'English' 'English' 'English' 'English' 'Mandarin' 'Mandarin']


In [ ]:
listy = label_csv['ID_Channels']
print len(listy)

In [ ]:
## 4/6 /let's try this again

baseline_files = []
part1 = []
part2 = []

missingb = []
missing = 0

for item in allWavs:
    fname = item
    searched = re.search('.*_(ch\d).*', item)
    channel = searched.group(1)
    if re.search("p(\d+)p(\d+)-(.*)_(ch\d)", item):
        nonBases = re.search("p(\d+)p(\d+)-(.*)_(ch\d)", item)
        id1, id2, part, testchan = nonBases.group(1), nonBases.group(2), nonBases.group(3), nonBases.group(4)
        #print id1, id2, part, testchan
        baseline = False
        trainf = True
    
    elif re.search('p(\d+).*', item):
        base = re.search('p(\d+).*', item)
        id1 = base.group(1)
        baseline = True
        trainf = False
        #print id1
    

    if baseline == True:
        #print baseline, fname
        for element in label_csv["ID_Channels"]:
            if type(element)== str:
                fname = "NA"
                pass
            elif type(element) == tuple:
                ID = element[0]
                chan = element[1]
                if id1 == ID:
                    print fname
    else:
        print "FALSE"


            
        #compare id1

print len(baseline_files)

    


In [ ]:
##print allWavs[:10]

###### NONE OF THIS WORKS AS OF 3/29/16

### Code that makes lists that correspond with the dataframe for baseline, part1, and part2s
baseline = []
part1 = []
part2 = []
count = 0

for i in range(len(label_csv['ID_Channels'])):
    tup = label_csv['ID_Channels'][i]
    #print tup
    if type(tup) == tuple:
        ID = tup[0]
        ch = tup[1]

        for item in allWavs:
            baseline_file = False
            searched = re.search('.*_(ch\d).*', item)
            channel = searched.group(1)
            if re.search("p(\d+)p(\d+)-(.*)_(ch\d)", item):
                nonBases = re.search("p(\d+)p(\d+)-(.*)_(ch\d)", item)
                id1, id2, part, testchan = nonBases.group(1), nonBases.group(2), nonBases.group(3), nonBases.group(4)
                #print id1, id2, part, testchan
                nonbaseFound = True
                baseFound = False
                #print item, baseFound, nonbaseFound




            elif re.search('p(\d+).*', item):
                base = re.search('p(\d+).*', item)
                id1 = base.group(1)
                baseFound = True
                nonbaseFound = False
                #print item, baseFound, nonbaseFound



              
            if nonbaseFound == True:
                if ch== testchan:
                    if ID == id1 or ID == id2:
                        nonbaseFound = True
                        part = part
                        break
                else:
                    nonbaseFound = False
                    break


        #print baseFound, nonbaseFound, ID, part
        #print "here"
        print baseFound, nonbaseFound, item
        if baseFound == False and nonbaseFound == False:
            entry = "NA"
            baseline.append(entry)
            part1.append(entry)
            part2.append(entry)
            test = 1
            print test            
            
        elif baseFound == False and nonbaseFound == True:
            entry = item
            empty = "NA"
            if part[-1] == 1:
                baseline.append(empty)
                part1.append(entry)
                part2.append(empty)
                test = 3
                print test
            elif part[-1] == 2:
                baseline.append(empty)
                part1.append(empty)
                part2.append(entry)
                test=4
                print test
                        
            
        elif baseFound == True and nonbaseFound == False:
            entry = item
            empty = "NA"
            baseline.append(entry)
            part1.append(empty)
            part2.append(empty)
            test = 2
            print test
        

    else:
        baseline.append("NA")
        part1.append("NA")
        part2.append("NA")
        test = 5
        print test
    count = count+1
    #print count
    print len(baseline), len(part1)


            
 

#print len(baseline)
#print len(part1)
#print len(part2)



In [ ]:
label_csv["baselineFiles"] = baseline
print label_csv